In [ ]:
import json
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def load_dakota_data():
    """
    Load Dakota Sobol indices from JSON files.

    Returns:
    --------
    dict with keys:
        'E_main': array of main indices for E
        'E_total': array of total indices for E
        'nu_main': array of main indices for nu
        'nu_total': array of total indices for nu
        'samples': array of sample sizes
    """
    sample_sizes = [200, 500, 1000, 2000, 10000, 25000]
    data = {'samples': sample_sizes, 'E_main': [], 'E_total': [], 'nu_main': [], 'nu_total': []}

    for n in sample_sizes:
        filepath = f'./data_files/dakota_sobol_output/sobol_indices_sample_{n}.json'
        with open(filepath, 'r') as f:
            json_data = json.load(f)

        params = json_data['response_functions']['max_von_Mises_stress']['parameters']
        data['E_main'].append(params['E']['main'])
        data['E_total'].append(params['E']['total'])
        data['nu_main'].append(params['nu']['main'])
        data['nu_total'].append(params['nu']['total'])

    # Convert lists to numpy arrays
    #for key in ['E_main', 'E_total', 'nu_main', 'nu_total']:
    for key in ['samples', 'E_main', 'E_total', 'nu_main', 'nu_total']:
        data[key] = np.array(data[key])

    return data


In [ ]:
def load_pyapprox_data():
    """
    Load PyApprox Sobol indices from NPZ files.

    Returns:
    --------
    dict with keys:
        'E_main': array of main indices for E
        'E_total': array of total indices for E
        'nu_main': array of main indices for nu
        'nu_total': array of total indices for nu
        'samples': array of sample sizes
    """
    sample_sizes = [50, 500, 1000, 5000, 10000]
    data = {'samples': sample_sizes, 'E_main': [], 'E_total': [], 'nu_main': [], 'nu_total': []}

    for n in sample_sizes:
        filepath = f'./data_files/pyapprox_sobol_mc_output/pyapprox_mc_sobol_{n}.npz'
        npz_data = np.load(filepath)

        # main_effects shape: (3, 4), row 1 = E, row 2 = nu, col 3 = max_von_Mises_stress
        # total_effects shape: (3, 4), same layout
        data['E_main'].append(npz_data['main_effects'][1, 3])
        data['E_total'].append(npz_data['total_effects'][1, 3])
        data['nu_main'].append(npz_data['main_effects'][2, 3])
        data['nu_total'].append(npz_data['total_effects'][2, 3])

    # Convert lists to numpy arrays
    #for key in ['E_main', 'E_total', 'nu_main', 'nu_total']:
    for key in ['samples', 'E_main', 'E_total', 'nu_main', 'nu_total']:
        data[key] = np.array(data[key])

    return data


In [ ]:
def create_plot1_overview(dakota_data, pyapprox_data):
    """
    Create overview plot with all 8 lines.

    Lines (8 total):
    1. E main - Dakota (dark orange, solid)
    2. E main - PyApprox (light orange, solid)
    3. E total - Dakota (dark orange, dashed)
    4. E total - PyApprox (light orange, dashed)
    5. nu main - Dakota (dark blue, solid)
    6. nu main - PyApprox (light blue, solid)
    7. nu total - Dakota (dark blue, dashed)
    8. nu total - PyApprox (light blue, dashed)
    """
    fig, ax = plt.subplots(figsize=(10, 6))

    # E indices - Dakota (dark orange)
    ax.plot(dakota_data['samples'], dakota_data['E_main'],
            color='#D97700', linestyle='-', marker='o', markersize=6,
            label='E main (Dakota)', linewidth=2)
    ax.plot(dakota_data['samples'], dakota_data['E_total'],
            color='#D97700', linestyle='--', marker='o', markersize=6,
            label='E total (Dakota)', linewidth=2)

    # E indices - PyApprox (light orange)
    ax.plot(pyapprox_data['samples'], pyapprox_data['E_main'],
            color='#FFB84D', linestyle='-', marker='s', markersize=6,
            label='E main (PyApprox)', linewidth=2)
    ax.plot(pyapprox_data['samples'], pyapprox_data['E_total'],
            color='#FFB84D', linestyle='--', marker='s', markersize=6,
            label='E total (PyApprox)', linewidth=2)

    # nu indices - Dakota (dark blue)
    ax.plot(dakota_data['samples'], dakota_data['nu_main'],
            color='#0066CC', linestyle='-', marker='o', markersize=6,
            label='nu main (Dakota)', linewidth=2)
    ax.plot(dakota_data['samples'], dakota_data['nu_total'],
            color='#0066CC', linestyle='--', marker='o', markersize=6,
            label='nu total (Dakota)', linewidth=2)

    # nu indices - PyApprox (light blue)
    ax.plot(pyapprox_data['samples'], pyapprox_data['nu_main'],
            color='#66A3FF', linestyle='-', marker='s', markersize=6,
            label='nu main (PyApprox)', linewidth=2)
    ax.plot(pyapprox_data['samples'], pyapprox_data['nu_total'],
            color='#66A3FF', linestyle='--', marker='s', markersize=6,
            label='nu total (PyApprox)', linewidth=2)

    ax.set_xscale('log')
    ax.set_xlabel('Number of Samples', fontsize=18)
    ax.set_ylabel('Sobol Index Value', fontsize=18)
    ax.set_title('Sobol Indices for max_von_Mises_stress: E and nu (All Data)', fontsize=18)
    #ax.legend(loc='best', fontsize=9, ncol=2)
    ax.legend(loc='center right', fontsize=12, ncol=2)
    ax.tick_params(axis='both', labelsize=14)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    #plt.savefig('sobol_indices_overview.png', dpi=300, bbox_inches='tight')
    #plt.savefig('sobol_indices_overview.pdf', bbox_inches='tight')


In [ ]:
dakota_data = load_dakota_data()

In [ ]:
pyapprox_data = load_pyapprox_data()

In [ ]:
create_plot1_overview(dakota_data,pyapprox_data)

In [ ]:
def create_plot2_E_zoom(dakota_data, pyapprox_data):
    """
    Zoomed plot focusing on E parameter with samples >= 1000.
    Tight y-axis range to show 1e-4 differences.
    """
    # Filter data for samples >= 1000
    #dakota_mask = dakota_data['samples'] >= 1000
    #pyapprox_mask = pyapprox_data['samples'] >= 1000

    dakota_mask = dakota_data['samples'] >= 0
    pyapprox_mask = pyapprox_data['samples'] >= 0
    fig, ax = plt.subplots(figsize=(12, 6))

    # E indices - Dakota
    ax.plot(dakota_data['samples'][dakota_mask], dakota_data['E_main'][dakota_mask],
            color='#D97700', linestyle='-', marker='o', markersize=7,
            label='E main (Dakota)', linewidth=2)
    ax.plot(dakota_data['samples'][dakota_mask], dakota_data['E_total'][dakota_mask],
            color='#D97700', linestyle='--', marker='o', markersize=7,
            label='E total (Dakota)', linewidth=2)

    # E indices - PyApprox
    ax.plot(pyapprox_data['samples'][pyapprox_mask], pyapprox_data['E_main'][pyapprox_mask],
            color='#FFB84D', linestyle='-', marker='s', markersize=7,
            label='E main (PyApprox)', linewidth=2)
    ax.plot(pyapprox_data['samples'][pyapprox_mask], pyapprox_data['E_total'][pyapprox_mask],
            color='#FFB84D', linestyle='--', marker='s', markersize=7,
            label='E total (PyApprox)', linewidth=2)

    # Calculate tight y-axis range
    all_E_values = np.concatenate([
        dakota_data['E_main'][dakota_mask], dakota_data['E_total'][dakota_mask],
        pyapprox_data['E_main'][pyapprox_mask], pyapprox_data['E_total'][pyapprox_mask]
    ])
    #y_min = np.min(all_E_values) - 0.01
    #y_max = np.max(all_E_values) + 0.01
    y_max = 1.03
    y_min = 0.978

    ax.set_ylim(y_min, y_max)
    ax.set_xlim(800,35000)

    ax.set_xscale('log')
    #ax.set_xlabel('Number of Samples', fontsize=18)
    #ax.set_ylabel('Sobol Index Value', fontsize=18)
    #ax.set_title('Sobol Indices for max_von_Mises_stress: E Parameter (≥1000 samples)', fontsize=18)
    #ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='both', labelsize=18,length=8, which="both", width=2)
    ax.hlines(1.0,0,100000,colors="black")

    plt.tight_layout()
    #plt.savefig('sobol_indices_E_zoom.png', dpi=300, bbox_inches='tight')
    #plt.savefig('sobol_indices_E_zoom.pdf', bbox_inches='tight')


In [ ]:
create_plot2_E_zoom(dakota_data,pyapprox_data)

In [ ]:
dakota_data["samples"]

In [ ]:
def create_plot3_nu_zoom(dakota_data, pyapprox_data):
    """
    Zoomed plot focusing on nu parameter with samples >= 1000.
    Tight y-axis range to show 1e-4 differences.
    """
    # Filter data for samples >= 1000
    #dakota_mask = dakota_data['samples'] >= 1000
    #pyapprox_mask = pyapprox_data['samples'] >= 1000
    dakota_mask = dakota_data['samples'] >= 0
    pyapprox_mask = pyapprox_data['samples'] >= 0

    fig, ax = plt.subplots(figsize=(12, 6))

    # nu indices - Dakota
    ax.plot(dakota_data['samples'][dakota_mask], dakota_data['nu_main'][dakota_mask],
            color='#0066CC', linestyle='-', marker='o', markersize=7,
            label='nu main (Dakota)', linewidth=2)
    ax.plot(dakota_data['samples'][dakota_mask], dakota_data['nu_total'][dakota_mask],
            color='#0066CC', linestyle='--', marker='o', markersize=7,
            label='nu total (Dakota)', linewidth=2)

    # nu indices - PyApprox
    ax.plot(pyapprox_data['samples'][pyapprox_mask], pyapprox_data['nu_main'][pyapprox_mask],
            color='#66A3FF', linestyle='-', marker='s', markersize=7,
            label='nu main (PyApprox)', linewidth=2)
    ax.plot(pyapprox_data['samples'][pyapprox_mask], pyapprox_data['nu_total'][pyapprox_mask],
            color='#66A3FF', linestyle='--', marker='s', markersize=7,
            label='nu total (PyApprox)', linewidth=2)

    # Calculate tight y-axis range
    all_nu_values = np.concatenate([
        dakota_data['nu_main'][dakota_mask], dakota_data['nu_total'][dakota_mask],
        pyapprox_data['nu_main'][pyapprox_mask], pyapprox_data['nu_total'][pyapprox_mask]
    ])
    #y_min = np.min(all_nu_values) - 0.0001
    #y_max = np.max(all_nu_values) + 0.0001
    y_min = -0.00008
    y_max = 0.0002
    ax.set_ylim(y_min, y_max)
    ax.set_xlim(800,35000)

    ax.set_xscale('log')
    #ax.set_xlabel('Number of Samples', fontsize=12)
    #ax.set_ylabel('Sobol Index Value', fontsize=12)
    #ax.set_title('Sobol Indices for max_von_Mises_stress: nu Parameter (≥1000 samples)', fontsize=14)
    #ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='both', labelsize=18,length=8, which="both", width=2)
    ax.hlines(0.0,0,100000,colors="black")

    plt.tight_layout()
    plt.savefig('sobol_indices_nu_zoom.png', dpi=300, bbox_inches='tight')
    plt.savefig('sobol_indices_nu_zoom.pdf', bbox_inches='tight')


In [ ]:
create_plot3_nu_zoom(dakota_data,pyapprox_data)